TỔNG QUAN VỀ BỘ DỮ LIỆU COVID-19 DATA

Mục lục

Định nghĩa vấn đề
 1.1 Bối cảnh dữ liệu (Johns Hopkins CSSE COVID-19)
 1.2 Mục tiêu của Notebook này

Chuẩn bị vấn đề
 2.1 Import thư viện
 2.2 Tải toàn bộ dữ liệu COVID-19

Tổng quan & thông tin các bảng
 3.1 Cấu trúc dữ liệu (ERD – conceptual)
 3.2 Thông tin chi tiết 3 bảng: Confirmed – Deaths – Recovered

Thống kê mô tả
 4.1 Confirmed Cases
 4.2 Deaths
 4.3 Recovered
 (hiển thị thông tin, kiểm tra thiếu – trùng lặp – chất lượng dữ liệu – xử lý cột ngày – logic thời gian)

Hiểu dữ liệu từ góc nhìn dịch tễ học
 5.1 Ý nghĩa các bảng dữ liệu
 5.2 Các biến quan trọng cho phân tích
 5.3 Mục tiêu phân tích dịch tễ

1. Định nghĩa vấn đề
1.1 Bối cảnh dữ liệu (Johns Hopkins CSSE)

Bộ dữ liệu được thu thập bởi Trung tâm Khoa học & Kỹ thuật Hệ thống – Johns Hopkins University (CSSE JHU).
Đây là một trong những nguồn dữ liệu uy tín nhất được sử dụng toàn cầu trong theo dõi COVID-19.

Dữ liệu bao gồm:

Số ca nhiễm tích lũy theo ngày (Confirmed)

Số ca tử vong tích lũy theo ngày (Deaths)

Số ca hồi phục tích lũy theo ngày (Recovered)

Trải dài trên hơn 180 quốc gia

Chi tiết đến mức tỉnh/bang tùy nước

Dữ liệu dạng Time Series nên rất phù hợp cho phân tích xu hướng, dự báo, và mô hình dịch tễ.

1.2 Mục tiêu của Notebook này

Notebook này nhằm:

Hiểu rõ cấu trúc dữ liệu COVID-19

Kiểm tra chất lượng dữ liệu (missing, duplicate, vấn đề thời gian)

Chuẩn hoá dữ liệu cho phân tích

Xây dựng nền tảng cho các phân tích sau:
✔ Xu hướng dịch theo quốc gia
✔ So sánh tỷ lệ tử vong
✔ R0, tốc độ tăng trưởng
✔ Dự báo theo chuỗi thời gian

2. Chuẩn bị vấn đề

2.1 Import thư viện

import pandas as pd
import numpy as np


2.2 Nạp dữ liệu

In [ ]:
confirmed = pd.read_csv("/mnt/data/time_series_covid19_confirmed_global.csv")
deaths = pd.read_csv("/mnt/data/time_series_covid19_deaths_global.csv")
recovered = pd.read_csv("/mnt/data/time_series_covid19_recovered_global.csv")


3. Tổng quan & thông tin các bảng

3.1 ERD khái niệm

   +------------------+
   |  Confirmed Cases |
   +------------------+
         | Country, Province
         |
         v
   +------------------+
   |     Deaths       |
   +------------------+
         | Country, Province
         |
         v
   +------------------+
   |    Recovered     |
   +------------------+


Cả ba bảng đều liên kết qua:

Province/State

Country/Region

Lat, Long

3.2 Thông tin chung các bảng

| Bảng          | Ý nghĩa                     | Hình dạng   | Ghi chú                              |
| ------------- | --------------------------- | ----------- | ------------------------------------ |
| **Confirmed** | Ca nhiễm tích lũy theo ngày | (289, 1147) | Full dữ liệu                         |
| **Deaths**    | Ca tử vong tích lũy         | (289, 1147) | Rất đồng nhất với Confirmed          |
| **Recovered** | Ca hồi phục tích lũy        | (274, 1147) | *Ít hơn do nhiều nước không báo cáo* |


4. Thống kê mô tả ban đầu

4.1 Confirmed Cases

✔ (1) Hiển thị thông tin cơ bản

Kích thước dữ liệu

In [ ]:
print(f"+ Kích thước dữ liệu: {confirmed.shape}")


Loại dữ liệu

In [ ]:
print(confirmed.dtypes)


Mẫu dữ liệu

In [1]:
display(confirmed.head(2))


NameError: name 'confirmed' is not defined

Thông tin dữ liệu

In [ ]:
confirmed.info()


Nhận xét:

Tổng số dòng: 289

Không có giá trị thiếu (ở phần định danh)

Số lượng cột ngày rất lớn (1143 cột)

Một số quốc gia có nhiều tỉnh (Canada, China, Australia)

✔ (2) Kiểm tra tính toàn vẹn dữ liệu

In [ ]:
has_null = confirmed.isnull().sum().any()
dup = confirmed.duplicated().sum()

print(has_null, dup)


Nhận xét:

Không có dòng trùng

Không có giá trị Null trong phần định danh

Số liệu ngày đầy đủ

4.2 Deaths

✔ (1) Hiển thị thông tin

Cấu trúc giống Confirmed

Không có dòng trùng

Không có missing tại phần định danh

✔ (2) Kiểm tra logic thời gian (với dữ liệu tích lũy)

Dữ liệu tử vong không bao giờ được giảm
→ có thể kiểm tra:
Nếu > 0 ⇒ có quốc gia điều chỉnh lại số ca.

In [ ]:
(deaths.iloc[:, 4:].diff(axis=1) < 0).sum().sum()


4.3 Recovered

✔ (1) Hiển thị thông tin

Có ít dòng hơn (274) → thiếu dữ liệu ở nhiều nước

Một số dòng có toàn giá trị 0

✔ (2) Kiểm tra missing

In [ ]:
recovered.isnull().sum().sum()


Nhận xét:

Recovered không đầy đủ, một số nước không báo cáo

Khi phân tích tỷ lệ hồi phục cần chú ý vấn đề này

5. Hiểu dữ liệu từ góc nhìn dịch tễ học

5.1 Ý nghĩa các bảng dữ liệu
| Bảng           | Vai trò                                       |
| -------------- | --------------------------------------------- |
| Confirmed      | Theo dõi tốc độ bùng phát, mô hình SIR, R0    |
| Deaths         | Tính tỷ lệ tử vong (CFR), mức độ nghiêm trọng |
| Recovered      | Đánh giá khả năng hồi phục, áp lực y tế       |
| Province/State | Phân tích theo vùng (tỉnh/bang)               |
| Lat/Long       | Vẽ bản đồ Heatmap, choropleth                 |


5.2 Các biến quan trọng cho phân tích dịch

Daily New Cases

Daily Deaths

Growth Rate

Death Rate / CFR = Deaths / Confirmed

Recovery Rate = Recovered / Confirmed

Peak detection

7-day rolling mean

5.3 Mục tiêu phân tích

Theo dõi mức độ lây lan COVID-19 theo thời gian

Xác định quốc gia chịu ảnh hưởng nặng nhất

Tính toán tỷ lệ tử vong theo quốc gia

So sánh tốc độ tăng trưởng giữa các vùng

Phân tích sự thay đổi xu hướng trước/ sau vaccine

Chuẩn bị dữ liệu cho mô hình dự báo

KẾT THÚC